In [1]:
import numpy as np
import sys, os, h5py, re
import matplotlib.pyplot as plt
from learning_utils import prepare_data_via, calc_loss_spl_via, spline_reconstruct_via
from check_dyn import plot_res

In [2]:
# just for dyna error calc
from casadi_kinodynamics.utils import symbolic_robot
sym_robot = symbolic_robot.symbolic_robot(robot_name='IRB910_2r', tool_mass=0.0,
                                                tool_frame=[0., 0., 0.3, 0., 0., 0.], load_casadi_fnc=False)

In [3]:
from algpr.gpr import GaussianProcessRegressor
from algpr.gpoegpr import GpoeGPR
from algpr.kernels import RBF as rbf
from trajencoder.bspline.bspline import CubicSpline, Order5Spline, QuinticSpline, Order7spline, Order8spline
from trajencoder.flexdmp.flexdmp import FlexDMP

In [4]:
SPL_KNOTS = 20
DMP_ORDER = 25

## Full gpr

The rate of dyna violation:  59.25925925925925 %  
The peak dyna violation:  [33.38533121  0.        ]  
Peak dyna violation occurs at:  [38.  0.]  
Time error: 0.0055, Shape error: 0.047, Dyna error: 0.7426565

In [5]:
X, Y, _ = prepare_data_via("../data/training/2-dof-no-velocity/training_SPL40_D8.hdf5", spline=QuinticSpline, knots=SPL_KNOTS)

In [6]:
X.shape

(3136, 4)

In [7]:
gpr = GaussianProcessRegressor(kernel=rbf(l=np.array([2.]*4), anisotropic=True))
gpr.fit(X, Y, call_hyper_opt=False)

Xv, Yv, Trueth, qd_trueth, qdd_trueth = \
        prepare_data_via("../data/training/2-dof-no-velocity/training_SPL40_D4.hdf5", spline=QuinticSpline, return_dyddy=True, knots=SPL_KNOTS)

The evidence is:  -20099160926821.28


In [11]:
pred = gpr.predict(Xv)
pred = Yv
error_time, error_with_q, error_dyna = calc_loss_spl_via(pred, Xv, Yv, Trueth, Order8spline, SPL_ORDER=SPL_KNOTS, \
            return_dyn_loss=True, tau_func=sym_robot.ck.get_inverse_dynamics_rnea(), tau_max=sym_robot.tau_max)
print("Time error: {0:.4f}, Shape error: {1:.3f}, Dyna error: {2:.7f}".format(error_time, error_with_q, error_dyna))

The rate of dyna violation:  58.333333333333336 %
The peak dyna violation:  [11.00767716  0.        ]
Peak dyna violation occurs at:  [129.   0.]
Time error: 0.0000, Shape error: 0.000, Dyna error: 0.1626152


## Gpoe gpr

In [12]:
gpr_ = GpoeGPR(kernel=rbf(l=np.array([2.1]*4), anisotropic=True), max_subset_num=1700)
gpr_.fit(X, Y, style='local', call_hyper_opt=False)

Xv, Yv, Trueth, qd_trueth, qdd_trueth = \
        prepare_data_via("../data/training/2-dof-no-velocity/training_SPL40_D4.hdf5", spline=QuinticSpline, return_dyddy=True, knots=SPL_KNOTS)

pred_, var_ = gpr_.predict(Xv)
error_time, error_with_q, error_dyna = calc_loss_spl_via(pred_, Xv, Yv, Trueth, Order8spline, SPL_ORDER=SPL_KNOTS, \
            return_dyn_loss=True, tau_func=sym_robot.ck.get_inverse_dynamics_rnea(), tau_max=sym_robot.tau_max)
print("Time error: {0:.4f}, Shape error: {1:.3f}, Dyna error: {2:.7f}".format(error_time, error_with_q, error_dyna))

The evidence is:  -6947790864094.85
The evidence is:  -6943669366535.346
Fitting finished, there are 2 gpr regressor were created!
The rate of dyna violation:  53.47222222222222 %
The peak dyna violation:  [38.6859351  0.       ]
Peak dyna violation occurs at:  [26.  0.]
Time error: 0.0034, Shape error: 0.046, Dyna error: 0.6293808


### Running speed test

In [13]:
from time import time
start = time()
for _ in range(100):
    pred_, var_ = gpr_.predict(Xv[0:1,:])
end = time()
print((end - start)*10, 'ms', 'per run.')

1.5186214447021484 ms per run.


In [14]:
pred_.shape

(1, 41)

In [15]:
tau_func = sym_robot.ck.get_inverse_dynamics_rnea()

for index in range(0, len(pred)):
    plot_res(tau_func, sym_robot.tau_max, sym_robot.q_max, sym_robot.q_min, sym_robot.qd_max, sym_robot.qdd_max,\
                 index, pred, Xv, Yv, Trueth, qd_trueth, qdd_trueth, spline=Order7spline, SPL_KNOTS=SPL_KNOTS)


KeyboardInterrupt



Error in callback <function flush_figures at 0x7f897ef99d30> (for post_execute):



KeyboardInterrupt

